In [1]:
from community_knapsack import *
from evaluation import evaluation
import random

In [2]:
# The algorithms to use for this evaluation:
exact_algorithms = [
    PBMultiAlgorithm.MEMOIZATION,
    PBMultiAlgorithm.DYNAMIC_PROGRAMMING,
    PBMultiAlgorithm.ILP_SOLVER
]

approximation_algorithms = [
    PBMultiAlgorithm.GREEDY,
    PBMultiAlgorithm.RATIO_GREEDY,
    PBMultiAlgorithm.BRANCH_AND_BOUND,
    PBMultiAlgorithm.SIMULATED_ANNEALING,
    PBMultiAlgorithm.GENETIC_ALGORITHM
]

#### This notebook tests the scalability of the algorithms with an increasing number of dimensions with small budgets and small costs.
---

In [3]:
# The boundaries of this evaluation:

# We test between one and ten dimensions where each budget is randomly generated
# between 10 and 1000 and the cost of each project in a dimension must be within
# 5-50% of the budget in that dimension.

random.seed(181)

num_projects = 50
num_voters = 3000

start_dimensions = 1
end_dimensions = 10
step_dimensions = 1

budget = [random.randint(10, 1000) for _ in range(end_dimensions//step_dimensions)]
cost_bounds = [
    (int(b * 0.05), int(b * 0.1))
    for bid, b in enumerate(budget)
]

In [4]:
x_axis = list(range(start_dimensions, end_dimensions + 1, step_dimensions))

In [5]:
problems = evaluation.generate_multi_problems(
    num_project_bounds=[(num_projects, num_projects)],
    num_voters_bounds=[(num_voters, num_voters)],
    dimension_bounds=x_axis,
    budget_bounds=[[[(b, b) for b in budget[:d]]] for d in x_axis],
    cost_bounds=[[[cost_bound for cost_bound in cost_bounds[:d]]] for d in x_axis]
)

In [6]:
# Obtain the exact results for this evaluation:
exact_results = evaluation.solve_problems(
    problem_list=problems,
    algorithms=exact_algorithms,
    timeout=120,
    max_fail=1,
    file_name='ext-small-dimensions-costs.json',
    output=True
)

Processing MEMOIZATION:
0
1
2
Processing DYNAMIC_PROGRAMMING:
0


/Users/finjordan/Desktop/Final Year Project/community-knapsack/community_knapsack/pbproblem.py:114: UserWarning: The MEMOIZATION algorithm did not finish within the 120 second timeout limit. Try increasing the timeout or using a different algorithm (such as an approximation scheme).
  warnings.warn(f'The {algorithm.name} algorithm did not finish within the {timeout} second timeout limit. '


1
Processing ILP_SOLVER:
0


/Users/finjordan/Desktop/Final Year Project/community-knapsack/community_knapsack/pbproblem.py:114: UserWarning: The DYNAMIC_PROGRAMMING algorithm did not finish within the 120 second timeout limit. Try increasing the timeout or using a different algorithm (such as an approximation scheme).
  warnings.warn(f'The {algorithm.name} algorithm did not finish within the {timeout} second timeout limit. '


1
2
3
4
5
6
7
8
9


Process Process-15:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/finjordan/Desktop/Final Year Project/community-knapsack/community_knapsack/pbproblem.py", line 255, in _worker
    allocation, value = algorithm(self.budget, self.costs, values)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/finjordan/Desktop/Final Year Project/community-knapsack/community_knapsack/pbalgorithm.py", line 107, in __call__
    return self.value[0](*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/finjordan/Desktop/Final Year Project/community-knapsack/community_knapsack/solvers/exact/

In [ ]:
# Obtain the exact results for this evaluation:
approximation_results = evaluation.solve_problems(
    problem_list=problems,
    algorithms=exact_algorithms,
    timeout=120,
    max_fail=1,
    file_name='apx-small-dimensions-costs.json',
    output=True
)